## Part I: Problem Solving

### 1 (b) 

In [ ]:
import scipy.stats as ss

In [ ]:
dice_toss = ss.binom(100,1.0/6.0)

In [ ]:
lower_bound,upper_bound = dice_toss.ppf(0.025),dice_toss.ppf(0.975)

In [ ]:
print lower_bound, upper_bound

### 1(c) . Given the following number of rolls until the dice become biased, can you conclude that it is 95% probable that aluminum performs better? (hint: may assume rolls until biased is well approximated as a Normal).
aluminum rolls until biased = [136, 73, 118, 122, 114, 103, 149, 118, 113, 105]
plastic rolls until biased = [129, 89, 97, 94, 124, 77, 85, 86, 86, 69]


In [ ]:
aluminium_rolls = [136, 73, 118, 122, 114, 103, 149, 118, 113, 105]
plastic_rolls = [129, 89, 97, 94, 124, 77, 85, 86, 86, 69]

In [ ]:
import numpy as np

In [ ]:
mean1, mean2 = np.mean(aluminium_rolls), np.mean(plastic_rolls)

In [ ]:
print mean1, mean2

In [ ]:
s1, s2 = np.std(aluminium_rolls), np.std(plastic_rolls)
print s1, s2

In [ ]:
n1 = n2 = 10
df1,df2 = n1-1,n2-1

In [ ]:
t = (float)(mean1-mean2)/np.sqrt((s1**2/n1) + (s2**2)/n2)

In [ ]:
print t

In [ ]:
p = 1 - ss.t.cdf(np.abs(t), df1+df2)

In [ ]:
print p

In [ ]:
ss.ttest_ind(aluminium_rolls,plastic_rolls)

### 2 (e)

In [ ]:
import numpy

from numpy import matrix

X = matrix ([[-1,2,1],[4,12,1],[3,5,1],[4,6,1],[-3,9,1],[6,7,1]])

R = ((X.T) * X).I * (X.T)

Y = matrix ([[2.5],[10],[8.5],[4],[1.5],[14]])
out = R* Y

print out


### 2 (f)

In [ ]:
import math
from __future__ import division
beta_0 = 0.0
beta_p = 0.0
left_handed = [0, 0, 1, 0, 0, 1]
plus_minus = [-1,4,3,4,-3,6]

diag = [0]*6
z = [0]*6
I = ([1]*6)
X = np.matrix([I, plus_minus]).T
#print X
while(1):
    prev_beta_0 = beta_0
    prev_beta_p = beta_p
    for i in range(6):
        q = math.exp(beta_0 + plus_minus[i]*beta_p)
        p = q/(1+q)
        diag[i] = p*(1-p)
        z[i] = math.log(p/(1-p)) + ((left_handed[i] - p)/(p*(1-p)))
    W = np.matrix(np.diag(diag))
    #print W
    Z = np.matrix(z).T
    #print Z
    beta = (((X.T)*W*X).I)*(X.T)*W*(Z)
    beta_p = beta.item(1)
    beta_0 = beta.item(0)
    #print beta
    if (round(beta_p,2) - round(prev_beta_p,2) < 0.01):
        break
print beta

In [ ]:
X = np.matrix ([[-1, 1],[4,1],[3,1],[4,1],[-3,1],[6,1]])
X_t = X.T
W = np.matrix(np.diag([0.25, 0.25, 0.25,0.25,0.25,0.25]))
z = np.matrix([[-2],[-2],[2],[-2],[-2],[2]])
r = X_t*W
r = (r*X).I
q = r*X_t
s = q*W
result = s*z
print result

# Part II: Programming

### 1. Read and tokenize the corpus.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import csv
import scipy.stats as ss
from happierfuntokenizing import Tokenizer
from BeautifulSoup import BeautifulSoup as Soup
from __future__ import division
import traceback
import json
import re
import operator
import math
from multiprocessing.pool import ThreadPool
import threading
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
#from threading import Thread
#from multiprocessing import Pool
#import multiprocessing

In [2]:
def process_blogs(path,filename,users,global_words_dict,industry_map,topics):
    tokenizer = Tokenizer()
    regex = r'<post>[\s\S]*?</post>'
    total_blog_posts = 0
    
    parts = filename.split(".")
    #print parts
    features_dict = {}
    words_dict ={}
    user_total_words = 0

    user_id = (int)(parts[0])
    gender = parts[1]
    age = (int)(parts[2])
    industry = parts[3]
    star_sign = parts[4]

    if user_id in users:
        features_dict = users[user_id]

    if industry in industry_map:
        tmp_count = industry_map[industry]
        tmp_count = tmp_count + 1
        industry_map[industry] = tmp_count
    else:
        industry_map[industry] = 1
            
    with open(path + filename, 'r') as user_blog:
        user_blogs = user_blog.read().replace('\n', '').replace('\r', '').replace('\t', '')
    
    blog_posts = re.findall(regex, user_blogs, re.DOTALL)

    total_blog_posts = total_blog_posts + len(blog_posts)
    user_total_words = 0
    #for blog in soup.findAll('post'):
    for blog in blog_posts:  
        words = tokenizer.tokenize(blog.strip())
        user_total_words += len(words)

        if 'dict' in features_dict:
            words_dict = features_dict['dict']

        for word in words:
            if word in words_dict:
                count = words_dict[word]
                count = count + 1
                words_dict[word] = count
            else:
                words_dict[word] = 1

            if word in global_words_dict:
                count = global_words_dict[word]
                count = count + 1
                global_words_dict[word] = count
            else:
                global_words_dict[word] = 1

    topics_prob = {}

    for i in range(2000):
        prob = 0.0
        topic_dict = topics[topics['category'] == i]

        for row in topic_dict.itertuples():
            word = row[1]
            prob_topic_given_word = row[3]
            if word in words_dict:
                count_user_word = words_dict[word]
                prob_word_given_user = count_user_word/user_total_words

                cur = prob_topic_given_word * prob_word_given_user

                prob = prob + cur
        topics_prob[i] = prob

    features_dict['dict'] = words_dict
    features_dict['age'] = age
    features_dict['industry'] = industry
    features_dict['star_sign'] = star_sign
    features_dict['user_id'] = user_id
    features_dict['topics'] = topics_prob
    features_dict['total_count'] = user_total_words
    features_dict['gender'] = gender
    users[user_id] = features_dict
    #print user_id , "completed"
    #return total_blog_posts
    return (users, global_words_dict, industry_map, total_blog_posts)

In [3]:
def parse_blogs(path):
    users = {}
    global_words_dict = {}
    industry_map = {}
    total_users = 0
    total_blog_posts = 0
    iterations = 0
    topics = pd.read_csv('wwbpFBtopics_condProb.csv')
    pool = ThreadPool(processes=1)
    jobs = []
    for filename in os.listdir(path):
        iterations += 1
        if iterations > 500:
            break
        if iterations % 10 == 0:
            print "user %d" %iterations
        
        #t = Thread(target= process_blogs, args=(path,filename,users,industry_map,global_words_dict,topics,))
        #t.start()
        #t = multiprocessing.Process(target= process_blogs, args=(path,filename,users,industry_map,global_words_dict,topics,)) 
        #jobs.append(t)
        #t.start()
        #total_blog_posts += t.join()
    #for j in jobs:
        #j.join()
    return (users, global_words_dict, industry_map, total_blog_posts)

In [ ]:
users, all_dict, industry_map, num_blogs  = parse_blogs('blogs/')
print "1. a) posts: " ,num_blogs
print "1. b) users: " , len(users)
print "1. c) words: " , len(all_dict)
print "1. d) ", industry_map

In [ ]:
import datetime
print datetime.datetime.now()

### 2.  Calculate users’ probability of mentioning each topic (“topic usage”).

In [ ]:
topics = pd.read_csv('wwbpFBtopics_condProb.csv')
topic_map = {}

topic_map[463] = topics[topics["category"]==463]
topic_map[963] = topics[topics["category"]==963]
topic_map[981] = topics[topics["category"]==981]

user_ids = sorted(k for k in users)
lowest_user_ids = sorted(user_ids)[0:3]

print "2.a)"
for userid in lowest_user_ids:
    print "%d" %userid
    
    word_counts_map = users[userid]['dict']

    all_words_wc = 0
    
    for w,c in word_counts_map.iteritems():
        all_words_wc = all_words_wc + word_counts_map[w]
        
    #print word_counts_map
    
    for topic in topic_map:
        print topic," : "
        
        prob = 0
        topic_pd = topic_map[topic]
        
        #print topic_pd
        for index, row in topic_pd.iterrows():
            word = row[0]
            prob_topic_given_word = row[2]

            if word in word_counts_map:
                count_user_word = word_counts_map[word]
                prob_word_given_user = count_user_word / all_words_wc

                #print prob_topic_given_word , prob_word_given_user
                cur = prob_topic_given_word * prob_word_given_user

                prob = prob + cur
            
        print prob

### 3 Correlate each topic usage with user age, adjusting for gender.

In [ ]:
beta_topics = {}
beta_gender = {}
beta_significant = {}
beta_p_value = {}

topics_user ={}

alpha = 0.05/2000

for topic in range(2000):
    topics = []
    gender = []
    Y_rows = []
    for user,features in users.iteritems():
        
        if topic in topics_user:
            user_topic = topics_user[topic]
        else:
            user_topic = {}
        
        user_topic[user] = features['topics'][topic]
        
        topics_user[topic] = user_topic
        
        topics.append(features['topics'][topic])
        gender.append(1 if features['gender'] == 'male' else 0)
        Y_rows.append([features['age']])
    try:
       
        topics = (topics - np.mean(topics))/np.std(topics)
        gender = (gender - np.mean(gender))/np.std(gender)
        Y_rows = (Y_rows - np.mean(Y_rows))/np.std(Y_rows)
        
        
        #break
        #X = np.matrix([topics,gender]).T
        X = []
        for i in range(len(topics)):
            X.append([topics[i],gender[i]])
        Y = []
        for i in range(len(topics)):
            Y.append([Y_rows[i]])
        X = np.matrix(X)
        Y = np.matrix(Y_rows)
        Y = (Y - np.mean(Y))/np.std(Y)
        '''print X.shape
        print Y.shape'''
        #break
        beta = (((X.T)*X).I)*(X.T)*Y
        #print beta
        beta_topics[topic] = beta.item(0)
        beta_gender[topic] = beta.item(1)
    except Exception as e:
        #print e, topic, user
        continue
    RSS = 0
    SE_topic = 0
    SE_gender = 0
    mean_topic = np.mean(topics)
    mean_gender = np.mean(gender)

    for i in range(len(users)):
        y_i = Y_rows[i][0]
        x_topic_i = topics[i]
        x_gender_i = gender[i]
        beta_topic_ = beta.item(0)
        beta_gender_ = beta.item(1)
        x_i = (beta_topic_*x_topic_i) + (beta_gender_*x_gender_i)
        
        RSS = RSS + ((y_i- x_i)**2)
        SE_topic =  SE_topic + ((x_topic_i - mean_topic)**2)
        #print y_i, x_i, RSS, SE_topic
    
    n = len(users)
    k = 2
    s_square = RSS/(n-k-1)
    t_topic = beta_topic_/(math.sqrt(s_square/SE_topic))
    p_value = ss.t.sf(np.abs(t_topic),len(users)-2)*2
    beta_p_value[topic] = p_value
    beta_significant[topic] = 'Yes' if p_value < alpha else 'No'

top_positive_topics = sorted(beta_topics.items(), key=operator.itemgetter(1),reverse = True)[:10]
top_negative_topics = sorted(beta_topics.items(), key=operator.itemgetter(1))[:10]
print '3.a)'
for topic, corr in top_positive_topics:
    print 'topic_id: %d, correlation: %0.3f, p-value: %f, signficant after correction? %s' \
    %(topic,corr,beta_p_value[topic], beta_significant[topic])
print '3. b)' 
for topic, corr in top_negative_topics:
    print 'topic_id: %d, correlation: %0.3f, p-value: %f, signficant after correction? %s' \
    %(topic,corr,beta_p_value[topic],beta_significant[topic])

### 4. Correlate each topic usage with user industry, adjusting for gender and age.

In [ ]:
industry_topic_beta = {}
for industry in industry_map:
    #if industry_map[industry] < 30:
        #continue
    topics_list = {}
    for topic in range(2000):
        beta_0 = 0.0
        beta_topic= 0.0
        beta_age = 0.0
        beta_gender = 0.0
        topics = []
        industry_row = []
        gender = []
        age = []
        constant = [1]*len(users)
        for user,features in users.iteritems():
            if 'topics' in features:
                topics.append(features['topics'][topic])
            else:
                topics.append(0)
            gender.append(1 if features['gender'] == 'male' else 0)
            age.append(features['age'])
            industry_row.append(0 if features['industry'] == industry else 1)
        
        age = (age - np.mean(age))/np.std(age)
        gender = (gender - np.mean(gender))/np.std(gender)
        topics = (topics - np.mean(topics))/np.std(topics)
        
        X = []
        for i in range(len(users)):
            X.append([topics[i],gender[i],age[i]])
        X = np.matrix(X)
       
        #X = np.matrix([topics,gender,age,constant]).T
        Y = np.matrix(industry_row).T
         
        try:
            while(1):
                prev_beta_0 = beta_0
                prev_beta_topic = beta_topic
                prev_beta_age = beta_age
                prev_beta_gender = beta_gender
                beta_matrix = np.matrix([beta_topic,beta_gender,beta_age,beta_0])
                diag = [0]*len(users)
                z = [0]*len(users)
            
                for i in range(len(users)):
                    q = math.exp(topics[i]*beta_topic + gender[i]*beta_gender + age[i]*beta_age)
                    p = q/(1+q)
                    diag[i] = p*(1-p)
                    z[i] = math.log(p/(1-p)) + ((industry_row[i] - p)/(p*(1-p)))
                    
                W = np.matrix(np.diag(diag))
                #print W
                Z = np.matrix(z).T
                #print Z
                beta = (((X.T)*W*X).I)*(X.T)*W*(Z)

                beta_topic = beta.item(0)
                beta_gender = beta.item(1)
                beta_age = beta.item(2)
                #print beta
                if (round(beta_topic,2) - round(prev_beta_topic,2) < 0.01):
                    break
        except Exception as e:
                print e
                break
                continue
        topics_list[topic] = beta_topic
    #print topics_list
    industry_topic_beta [industry] = topics_list

#print industry_topic_beta

In [ ]:
filtered_topics = []
print '4 a).'
for industry,topics_list in industry_topic_beta.iteritems():
    for topic,beta_topic in sorted(topics_list.items(), key=operator.itemgetter(1),reverse = True)[:5]:
        if topic not in filtered_topics:
            filtered_topics.append(topic)
        print 'industry: %s, topic_id: %d, coefficient: %f' %(industry,topic, beta_topic)
print '4 b).'
for industry,topics_list in industry_topic_beta.iteritems():
    for topic,beta_topic in sorted(topics_list.items(), key=operator.itemgetter(1))[:5]:
        if topic not in filtered_topics:
            filtered_topics.append(topic)
        print 'industry: %s, topic_id: %d, coefficient: %f' %(industry,topic, beta_topic)

### 5 Plot topics by Age

In [ ]:


topics_terms_pd = pd.read_csv("2000topics.top20freqs.keys.csv")

Topic_ages_mean = {}

colors =  matplotlib.colors.cnames.keys()[:len(users)]

for topic in filtered_topics:

    topic_users = topics_user[topic]
    
    top_topic_users = sorted(topic_users.items(),key=operator.itemgetter(1),reverse=True)

    ages = []
    num_top_users = int (round(0.25*len(topic_users)))

    for user,beta in top_topic_users[:num_top_users]:
        #print user
        ages.append(users[user]['age'])

    #print ages
    ages_mean = np.mean(ages)
    Topic_ages_mean[topic] = round(ages_mean,3)

fig = plt.figure(figsize=(15,15))
sub_plot = 0
for industry,topics_list in industry_topic_beta.iteritems():

    top5pairs = sorted(topics_list.items(), key=operator.itemgetter(1),reverse = True)[:5]
    bottom5pairs = sorted(topics_list.items(), key=operator.itemgetter(1))[:5]

    top5pairs.extend(bottom5pairs)
    #print top5pairs
    X = []
    Y = []
    topic_terms_map = {}
    for tid in top5pairs:
        
        words_list = []
        topic = tid[0]
        
        X.append(round(tid[1],3))
        Y.append(Topic_ages_mean[topic])
        
        topic_terms = topics_terms_pd.iloc[topic-1]
    
        #print topic_terms.values
        for wd in topic_terms.values:
            if isinstance(wd, str):
                if len(words_list) > 3:
                    break
                else:
                    words_list.append(wd)
                
        topic_terms_map[topic] = words_list
    
    tpc_words_list = topic_terms_map.values()
    
    fig.suptitle(industry, fontsize=14, fontweight='bold')
    sub_plot += 1 
    ax = fig.add_subplot(len(industry_topic_beta),1,sub_plot)
    fig.subplots_adjust(top=20)
    ax.set_title('Topic Correlation with %s Vs Mean Age of top 25 percent users' % (industry))

    ax.set_xlabel('Topic Correlation with %s '% (industry))
    ax.set_ylabel('Mean Age of top 25% users')
    
    for i in range(10):
        #for j in range(len(tpc_words_list[i])):
        ax.text(X[i], Y[i],'\n'.join([','.join(tpc_words_list[i][0:2]),','.join(tpc_words_list[i][2:])]),\
                 fontsize=12, color = colors[i])
        
    ax.axis([np.min(X),np.max(X)*1.5, np.min(Y), np.max(Y)])
#fig.tight_layout()
plt.savefig("out.png", transparent = True)
plt.show()
#plt.savefig("a1_4_histpop.png")
#plt.clf()

In [ ]:
p_value = ss.t.sf(np.abs(3.16),6-2)*2

In [ ]:
print p_value